# Initialize Environment

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
# Compute the drawdown of an array, in place.
def drawdown (array):
    dd=deepcopy(array)
    maxval=-1
    for i in range(len(dd)):
        idx = i
        if array[idx] > maxval: maxval = array[idx]
        dd[idx] = (array[idx]/maxval - 1.) * 100.
    return dd

# Load Data from Separate Notebook

In [3]:
%run data.ipynb

[*********************100%***********************]  7 of 7 completed


# Plot Data
## Vanguard

In [4]:
# Plot the prices & drawdowns over/under
def plot_vg_prices_dd(tickers, 
                      data,
                      logy,
                      date_min, 
                      date_max,
                      elapsed_days=-1):
        
    #print(data, len(data))
    print("vg_data_range={} to {}".format(data.index[0],
                                          data.index[-1]))

    if not date_max: date_max=datetime.date.today()
    if not date_min: date_min=date_max - datetime.timedelta(days=2*365)
    if elapsed_days > 1: date_min=date_max - datetime.timedelta(days=elapsed_days)
        
    tdelta = date_max - date_min
    print(date_min, date_max, tdelta)
    if tdelta > datetime.timedelta(days=50*365):
        print("requested time delta > 50 years, resetting")
        return

    fig, ax  = plt.subplots(nrows=2, ncols=1,
                            sharex=True, sharey=False,
                            figsize=(12,8*2))
    
    ax[0].set_title('Vanguard Fund Price, {} to {}'.format(date_min, date_max))
    ax[1].set_title('Vanguard Fund Drawdowns, {} to {}'.format(date_min, date_max))
    
    for t in tickers:
        #print(t, data[t].columns)
        #print(t, data[t])
        close = data['Close'][t]
        #print(type(close.index), type(close.values))
        dd = drawdown(close.values)

        ax[0].plot(close.index, close.values, label='{} (${:.2f})'.format(t,close.values[-1]), linewidth=2.0)
        ax[0].grid(visible=True, which='major', color='#999999', linestyle='-')
        ax[0].minorticks_on()
        ax[0].grid(visible=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
        ax[0].legend(fancybox=True, loc=2)
        if logy: ax[0].set_yscale('log')

        ax[1].plot(close.index, dd, label='{} ({:.1f}%)'.format(t,dd[-1]), linewidth=2.0)
        ax[1].grid(visible=True, which='major', color='#999999', linestyle='-')
        ax[1].minorticks_on()
        ax[1].grid(visible=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
        ax[1].legend(fancybox=True, loc=3)
        ax[1].set_xlim([date_min, date_max])
        plt.xticks(rotation=60)
    fig.savefig('vg_shareprices.pdf', bbox_inches='tight')
    return


w = widgets.SelectMultiple(
    options=tickers,
    value=tickers,
    #rows=10,
    description='Tickers',
    disabled=False)

#plot_prices_dd(tickers, data)
p = interact(plot_vg_prices_dd,
             tickers=w, data=fixed(data),
             logy=False,
             date_min=widgets.DatePicker(description='Start Date', disabled=False),
             date_max=widgets.DatePicker(description='End Date', disabled=False),
             elapsed_days=widgets.IntSlider(min=-1, 
                                            max=min(15*365,len(data)), 
                                            step=1, value=-1,  
                                            continuous_update=False))

interactive(children=(SelectMultiple(description='Tickers', index=(0, 1, 2, 3, 4, 5, 6), options=('VINIX', 'VS…

## TSP

In [5]:
# Plot the prices & drawdowns over/under
def plot_tsp_prices_dd(tsp_tickers,
                       tspdata,
                       logy,
                       date_min,
                       date_max,
                       elapsed_days=-1):
    
    #print(tspdata, len(tspdata))
    print("tsp_data_range={} to {}".format(tspdata.index[0],
                                           tspdata.index[-1]))

    if not date_max: date_max=datetime.date.today()
    if not date_min: date_min=date_max - datetime.timedelta(days=2*365)
    if elapsed_days > 1: date_min=date_max - datetime.timedelta(days=elapsed_days)
        
    tdelta = date_max - date_min
    print(date_min, date_max, tdelta)
    if tdelta > datetime.timedelta(days=50*365):
        print("requested time delta > 50 years, resetting")
        return
    
    fig, ax  = plt.subplots(nrows=2, ncols=1,
                            sharex=True, sharey=False,
                            figsize=(12,8*2))
    
    ax[0].set_title('TSP Fund Price, {} to {}'.format(date_min, date_max))
    ax[1].set_title('TSP Fund Drawdowns, {} to {}'.format(date_min, date_max))

   
    for c in tsp_tickers:
        if not c: continue
        close = tspdata[c]
        #print(type(close.index), type(close.values))
        #print(close)
        dd = drawdown(close.values)

        ax[0].plot(close.index, close.values, label='{} (${:.4f})'.format(c,close.values[-1]), linewidth=2.0)
        ax[0].grid(visible=True, which='major', color='#999999', linestyle='-')
        ax[0].minorticks_on()
        ax[0].grid(visible=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
        ax[0].legend(fancybox=True, loc=2)
        if logy: ax[0].set_yscale('log')

        #if 'G Fund' in c: continue
        ax[1].plot(close.index, dd, label='{} ({:.1f}%)'.format(c,dd[-1]), linewidth=2.0)
        ax[1].grid(visible=True, which='major', color='#999999', linestyle='-')
        ax[1].minorticks_on()
        ax[1].grid(visible=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
        ax[1].legend(fancybox=True, loc=3)
        ax[1].set_xlim([date_min, date_max])
        plt.xticks(rotation=60)
    fig.savefig('tsp_shareprices.pdf', bbox_inches='tight')
    return

if False:
    w = widgets.SelectMultiple(options=tuple(tspdata.columns),
                               value=tuple(tspdata.columns),
                               #rows=10,
                               description='Tickers',
                               disabled=False) 

    #plot_tsp_prices_dd(tspdata, ax)
    p = interact(plot_tsp_prices_dd,
                 tsp_tickers=w, tspdata=fixed(tspdata),
                 logy=False,
                 date_min=widgets.DatePicker(description='Start Date', disabled=False),
                 date_max=widgets.DatePicker(description='End Date',   disabled=False),
                 elapsed_days=widgets.IntSlider(min=-1, 
                                                max=len(tspdata), 
                                                step=1, value=-1,  
                                                continuous_update=False))

# Correlations
## Vanguard

In [6]:
import seaborn as sns
sns.set(style="white")

def vg_corr(num_days=None):
    # Set up the matplotlib figure
    fig, ax = plt.subplots(figsize=(10, 10))
    
    corr = ddata.iloc[-num_days:].corr() if num_days else ddata.corr()

    title = 'Vanguard Fund Correlations'
    if num_days: title += ' for Previous {} sessions'.format(num_days)
    
    ax.set_title(title)
    
    # Generate a mask for the upper triangle or diagonal
    #mask = np.triu(np.ones_like(corr, dtype=np.bool))
    mask = np.eye(len(corr))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, center=0, vmin=-1, vmax=1, cmap=cmap, annot=True,
                square=True, linewidths=.5, cbar_kws={"shrink": 0.5, "orientation": "horizontal"})

    ax.set_xlabel(None)
    ax.set_ylabel(None)
    #ax.set_yticklabels([ item.get_text().strip('Close-') for item in ax.get_yticklabels() ])
    ax.set_yticklabels(ax.get_yticklabels(),
                       rotation=0,
                       verticalalignment='center')
    ax.set_xticklabels(ax.get_yticklabels(),
                       rotation=45,
                       horizontalalignment='right')
    (bottom, top) = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    fig.savefig('vg_correlation.pdf', bbox_inches='tight')
    return


#p = interact(vg_corr, 
#             num_days=widgets.IntSlider(min=10, 
#                                        max=min(5*365,len(ddata)), 
#                                        step=1, value=100, 
#                                        continuous_update=False))
p = interact(vg_corr, 
             num_days=widgets.IntText(value=100, description='# Days'))

interactive(children=(IntText(value=100, description='# Days'), Output()), _dom_classes=('widget-interact',))

## TSP

In [7]:
def tsp_corr(num_days=None):
    # Set up the matplotlib figure
    fig, ax = plt.subplots(figsize=(10, 10))

    corr = tspdata.iloc[-num_days:].corr() if num_days else tspdata.corr()

    title = 'TSP Fund Correlations'
    if num_days: title += ' for Previous {} sessions'.format(num_days)
    
    ax.set_title(title)

    # Generate a mask for the upper triangle or diagonal
    #mask = np.triu(np.ones_like(corr, dtype=np.bool))
    mask = np.eye(len(corr))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, center=0., vmin=-1, vmax=1, cmap=cmap, annot=True,
                square=True, linewidths=.5, cbar_kws={"shrink": 0.5, "orientation": "horizontal"})


    ax.set_xlabel(None)
    ax.set_ylabel(None)
    #ax.set_yticklabels([ item.get_text().strip('Close-') for item in ax.get_yticklabels() ])
    ax.set_yticklabels(ax.get_yticklabels(),
                       verticalalignment='center')
    ax.set_xticklabels(ax.get_yticklabels(),
                       rotation=45,
                       horizontalalignment='right')
    (bottom, top) = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    fig.savefig('tsp_correlation.pdf', bbox_inches='tight')
    return

if False:
    p = interact(tsp_corr, 
                 num_days=widgets.IntText(value=100, description='# Days'))

# Same for ETFs, which can be run separately.

In [8]:
%run etfs.ipynb

[*********************100%***********************]  12 of 12 completed


interactive(children=(SelectMultiple(description='Tickers', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), opti…

interactive(children=(IntText(value=100, description='# Days'), Output()), _dom_classes=('widget-interact',))